# Lectures 4 & 5

## Links:
### https://www.math.cmu.edu/~af1p/Teaching/MCC17/Papers/KLM.pdf
### https://www.youtube.com/watch?v=k6qfHkUyvxM&list=PLOQjlWvnI0faRpH2oJcyW4CuM5Clt8a2n&index=4

## Karp-Luvy importance sampling for DNF counting

In [1]:
import random

### Generate instances function

In [2]:
def generateDNF(n=20, debug=False):
    # random variables are X1, X2, ..., Xn
    # m (number of clauses) is uniformly randomly sampled from [1, n^2]
    # i.e., m is O(n^2)
    # O(m^2n) time and O(mn) space, i.e., 
    # O(n^5) time and O(n^3) space
    # DNF = [[1,2],[-3,-4]] means
    # DNF = (X1 & X2) OR (not(X3) & not(X4))
    
    m = random.randint(1,n**2)
    
    if debug:
        print('m:', m)
    
    instance = []
    for _ in range(m):
        # sample k numbers from [1, n], where k is sampled uniformly randomly from [1, n]
        s = random.sample([i for i in range(1,n+1)], random.randint(1,n))
        s.sort()
        
        # we create the clause flipping the sign of each number in the sample randomly 
        clause = []
        for var in s:
            clause.append(var*(-1)**(random.randint(1,2)))
        
        if debug:
            print('pre-clause i-th:', s)
            print('clause i-th:', clause)
        
        # we only add new clauses to the instance
        if not(clause in instance):
            instance.append(clause)
                
    # to see how many clauses we dropped due repetition in instance
    print('m vs added:', m,len(instance))
        
    return instance

In [3]:
# TEST
# DNF = [[1,2],[-3,-4]] means
# DNF = (X1 & X2) OR (not(X3) & not(X4))
print(generateDNF(debug=False))

m vs added: 60 59
[[-1, -2, 3, -4, 5, -6, 7, 9, -11, 12, 13, -14, 15, 16, -18, 19, 20], [-2, 6, -7, -9, -10, 12, 16, 17, -19], [-1, 3, -5, -6, -7], [-2, -4, 9, 14, 16, -19], [3, 10], [2, 13, 17], [-3, -5, 9, -10, 11, 12, 14, -15, -17, 20], [-2, -5, 8, 10, -11, -12, 15, -18, 20], [-1, 2, 3, -4, -5, 6, -7, -8, 9, -10, -11, -12, 13, -14, 16, 17, 19, 20], [-2], [3, 6, -7, 10, 13, 15, 18, 19], [-1, -2, -4, 5, 6, 7, -8, -9, 10, -11, -12, 13, -14, 15, 16, 17, -18, -19, 20], [1, 3, -8, 10, 11, 12, 13, -14, -15, 18, -19, -20], [2, 3, 4, 5, 8, 10, -12, -14, 16, 17, 18, 19], [1, -3, 5, 8, -11, -12, -14, 15, -18, -19, -20], [1, 2, 5, 9, -10, -12, -17, -19, 20], [1, -2, -3, 5, -6, -7, 8, -13, -14, -15, 16, -17, 19, -20], [-1, 2, 3, 4, -5, 6, 8, -9, -10, -11, -12, 13, -14, 19, 20], [3, 4, -5, 6, 7, -10, -11, 12, -13, -14, -15, -16, 17, -18, -20], [-4, -5, 11, 13], [2, -4, -6, 7, 8, -9, -10, -11, 16, 19], [-1, 2, 3, 4, 5, -6, -7, 8, 9, 10, 11, 12, 13, 14, 16, -17, 18, -19, 20], [-1, -2, 12, 16, 19], 

## Solve instances function
### compute true solution
### (O(mn2^n)) which implies approx (O(n^3 * 2^n)) but as n=20, this is approx. 10^10, doable for a modern PC within seconds

In [4]:
def solveDNF(instance, n, debug=False):
    # brute force
    # O(mn 2^n) time and O(n) space, i.e., 
    # O(n^3 2^n) time and O(n) space
    
    # compute m
    m = len(instance)
    
    if debug:
        print('m:', m)
        
    # iterate through all possible assignments
    s = 0
    for j in range(2**n):
        # get binary representation of j
        j = str(bin(j))[2:]
        
        # generate assignment
        sa = [0]*(n-len(j))
        
        for b in j:
            sa.append(int(b))
                        
        if debug:
            print('binary assignment:', j)
        
        # check if assignment satisfies instance.
        # If it does, increment output s
        for clause in instance:
            sat = True
            
            # check if assignment satisfies every
            # variable in clause
            for b in clause:
                if b > 0:
                    if sa[b-1] == 0:
                        sat = False
                        break
                else:
                    if sa[-b-1] == 1:
                        sat = False
                        break
            
            # if assignment satisfies clause, update
            # output s and move to next assignment
            if sat:
                s += 1
                break
                
        if debug:
            print('current count: ', s)
                        
    return s

### Tests

In [5]:
# deterministic input, n=6, m=2, solution=28
# DNF = (X1 & X2) OR (not(X3) & not(X4))
# in its binary representation, X1 corresponds
# to the most significant bit and Xn corresponds
# to the least significant bit, i.e.,
# (X1,X2, ..., Xn)

instance0 = [[1,2],[-3,-4]]
sol0 = solveDNF(instance0, 6, debug=True)
print(sol0)

m: 2
binary assignment: 0
current count:  1
binary assignment: 1
current count:  2
binary assignment: 10
current count:  3
binary assignment: 11
current count:  4
binary assignment: 100
current count:  4
binary assignment: 101
current count:  4
binary assignment: 110
current count:  4
binary assignment: 111
current count:  4
binary assignment: 1000
current count:  4
binary assignment: 1001
current count:  4
binary assignment: 1010
current count:  4
binary assignment: 1011
current count:  4
binary assignment: 1100
current count:  4
binary assignment: 1101
current count:  4
binary assignment: 1110
current count:  4
binary assignment: 1111
current count:  4
binary assignment: 10000
current count:  5
binary assignment: 10001
current count:  6
binary assignment: 10010
current count:  7
binary assignment: 10011
current count:  8
binary assignment: 10100
current count:  8
binary assignment: 10101
current count:  8
binary assignment: 10110
current count:  8
binary assignment: 10111
current cou

In [6]:
# randomly generated instance
n = 20
instance1 = generateDNF()

m vs added: 193 193


In [7]:
sol1 = solveDNF(instance1, n, debug=False)
print(sol1)

1048576


## FPRAS (Fully Polynomial Randomized Approximation Scheme)
### Pr[output in (1+-eps)true_value] >= 2/3(?)

## KL Algorithm

In [8]:
def KL(instance, n, eps, debug=False):
    # O(m^2n/(eps^2)) time and O(m+n) memory

    # compute m and t
    m = len(instance)
    t = int(m/(eps**2))
    
    if debug:
        print('m:', m)
        print('eps:', eps)
        print('t:', t)
        
    # compute importances
    un_importances = []
    for clause in instance:
        l = len(clause)
        un_importances.append(2**(n-l))
        
    B = sum(un_importances)
    importances = []
    for imp in un_importances:
        importances.append(imp/B)
        
    if debug:
        print('B:', B)
        print('importances:', importances)
        
    # iterate t=m/(eps^2) times
    s = 0
    for j in range(t):
        # choose ith clause according to importance sampling
        index = random.choices([i for i in range(m)], weights=importances)[0]
        clause = instance[index]
        
        # choose a satisfying assignment of ith clause uniformly randomly
        sa = []
        for i in range(1, n+1):
            # if ith variable in clause, add a 1 to the satisfying assignment
            if i in clause:
                sa.append(1)
            # if negated ith variable in clause, add a 0 to the satisfying assignment
            elif -i in clause:
                sa.append(0)
            # if ith variable not in clause, add a 0 or 1 uar
            elif random.randint(1,2) == 1:
                sa.append(1)
            else:
                sa.append(0)
                
        
        # check if assignment satisfies any previous clause. If it does,
        # Xj=0, else Xj = 1
        for i in range(index):
            clause = instance[i]
            sat = True
            
            # check if assignment satisfies every
            # variable in clause
            for b in clause:
                if b > 0:
                    if sa[b-1] == 0:
                        sat = False
                        break
                else:
                    if sa[-b-1] == 1:
                        sat = False
                        break
            
            # if assignment satisfies clause, update
            # output s and break loop
            # i.e., Xj=0
            if sat:
                s -= 1
                break
                
        # compensate output
        s += 1
                
        if debug:
            print('------------------------')
            print('iteration:', j)
            print('clause index:', index)
            print('satisfactory assignment:', sa)
            print('count:', s)
        
    return s*B//t
        

### Tests

In [9]:
# deterministic input, n=6, m=2, solution=28
# DNF = (X1 & X2) OR (not(X3) & not(X4))

eps = 0.1
KL0 = KL(instance0, 6, eps, debug=True)

print('------------------------')
print('approximation:', KL0)
print('eps:', eps)
print('ratio (real eps):', (KL0/sol0)-1)

m: 2
eps: 0.1
t: 199
B: 32
importances: [0.5, 0.5]
------------------------
iteration: 0
clause index: 0
satisfactory assignment: [1, 1, 1, 0, 0, 1]
count: 1
------------------------
iteration: 1
clause index: 1
satisfactory assignment: [1, 1, 0, 0, 1, 1]
count: 1
------------------------
iteration: 2
clause index: 0
satisfactory assignment: [1, 1, 0, 1, 1, 1]
count: 2
------------------------
iteration: 3
clause index: 0
satisfactory assignment: [1, 1, 0, 0, 0, 0]
count: 3
------------------------
iteration: 4
clause index: 1
satisfactory assignment: [1, 0, 0, 0, 0, 0]
count: 4
------------------------
iteration: 5
clause index: 0
satisfactory assignment: [1, 1, 1, 0, 1, 1]
count: 5
------------------------
iteration: 6
clause index: 0
satisfactory assignment: [1, 1, 0, 1, 1, 0]
count: 6
------------------------
iteration: 7
clause index: 0
satisfactory assignment: [1, 1, 0, 1, 1, 0]
count: 7
------------------------
iteration: 8
clause index: 0
satisfactory assignment: [1, 1, 1, 0, 1

count: 133
------------------------
iteration: 150
clause index: 1
satisfactory assignment: [1, 0, 0, 0, 1, 1]
count: 134
------------------------
iteration: 151
clause index: 0
satisfactory assignment: [1, 1, 1, 0, 0, 1]
count: 135
------------------------
iteration: 152
clause index: 1
satisfactory assignment: [0, 0, 0, 0, 0, 1]
count: 136
------------------------
iteration: 153
clause index: 0
satisfactory assignment: [1, 1, 1, 0, 0, 0]
count: 137
------------------------
iteration: 154
clause index: 0
satisfactory assignment: [1, 1, 1, 0, 0, 1]
count: 138
------------------------
iteration: 155
clause index: 1
satisfactory assignment: [0, 1, 0, 0, 0, 0]
count: 139
------------------------
iteration: 156
clause index: 0
satisfactory assignment: [1, 1, 1, 1, 0, 1]
count: 140
------------------------
iteration: 157
clause index: 0
satisfactory assignment: [1, 1, 1, 1, 0, 1]
count: 141
------------------------
iteration: 158
clause index: 1
satisfactory assignment: [1, 0, 0, 0, 1, 0]
c

In [10]:
# randomly generated instance
KL1 = KL(instance1, n, eps, debug=False)

print('------------------------')
print('approximation:', KL1)
print('eps:', eps)
print('ratio (real eps):', (KL1/sol1)-1)

------------------------
approximation: 1040871
eps: 0.1
ratio (real eps): -0.007348060607910156


## KL improved Algorithm

In [11]:
def KLI(instance, n, eps, debug=False):
    # O(mn/(eps^2)) time and O(m+n) memory

    # compute m and t
    m = len(instance)
    t = int(m/(eps**2))
    
    if debug:
        print('m:', m)
        print('eps:', eps)
        print('t:', t)
        
    # compute importances
    un_importances = []
    for clause in instance:
        l = len(clause)
        un_importances.append(2**(n-l))
        
    B = sum(un_importances)
    importances = []
    for imp in un_importances:
        importances.append(imp/B)
        
    if debug:
        print('B:', B)
        print('importances:', importances)
    
    # repeat until t=m/(eps^2) operations
    c = 0
    ls = 0
    n_ls = 0
    while c < t: 
        # choose ith clause according to importance sampling
        index = random.choices([i for i in range(m)], weights=importances)[0]
        clause = instance[index]
        
        # choose a satisfying assignment of ith clause uniformly randomly
        sa = []
        for i in range(1, n+1):
            # if ith variable in clause, add a 1 to the satisfying assignment
            if i in clause:
                sa.append(1)
            # if negated ith variable in clause, add a 0 to the satisfying assignment
            elif -i in clause:
                sa.append(0)
            # if ith variable not in clause, add a 0 or 1 uar
            elif random.randint(1,2) == 1:
                sa.append(1)
            else:
                sa.append(0)
            
        if debug:
            print('------------------------')
            print('------------------------')
            print('clause index to compute satisfactory assignment:', index)
            print('satisfactory assignment:', sa)

        # iterate
        l = 0
        while True:
            l += 1
            c += 1

            # pick a clause uniformly randomly
            j = random.randint(0, m-1)
            
            if debug:
                print('------------------------')
                print('clause index:', j)
                print('l:', l)
                print('operations count:', c)
                
            # check if assignment satisfies j-th clause.
            # If it does, break (while True loop)
            sat = True
            for b in instance[j]:
                if b > 0:
                    if sa[b-1] == 0:
                        sat = False
                        break
                else:
                    if sa[-b-1] == 1:
                        sat = False
                        break

            if sat:
                break
    
        n_ls += 1
        ls += (l-ls)/n_ls
        
        if debug:
            print('ls:', ls)

    return ls*B//m
        

### Tests

In [12]:
# deterministic input, n=6, m=2, solution=28
# DNF = (X1 & X2) OR (not(X3) & not(X4))
KLI0 = KLI(instance0, 6, eps, debug=True)

print('------------------------')
print('approximation:', KLI0)
print('eps:', eps)
print('ratio (real eps):', (KLI0/sol0)-1)

m: 2
eps: 0.1
t: 199
B: 32
importances: [0.5, 0.5]
------------------------
------------------------
clause index to compute satisfactory assignment: 1
satisfactory assignment: [0, 1, 0, 0, 1, 0]
------------------------
clause index: 0
l: 1
operations count: 1
------------------------
clause index: 1
l: 2
operations count: 2
ls: 2.0
------------------------
------------------------
clause index to compute satisfactory assignment: 0
satisfactory assignment: [1, 1, 0, 1, 1, 1]
------------------------
clause index: 1
l: 1
operations count: 3
------------------------
clause index: 0
l: 2
operations count: 4
ls: 2.0
------------------------
------------------------
clause index to compute satisfactory assignment: 1
satisfactory assignment: [0, 1, 0, 0, 0, 0]
------------------------
clause index: 1
l: 1
operations count: 5
ls: 1.6666666666666667
------------------------
------------------------
clause index to compute satisfactory assignment: 0
satisfactory assignment: [1, 1, 1, 1, 0, 1]


satisfactory assignment: [1, 1, 0, 0, 0, 1]
------------------------
clause index: 0
l: 1
operations count: 167
ls: 1.7765957446808514
------------------------
------------------------
clause index to compute satisfactory assignment: 1
satisfactory assignment: [0, 1, 0, 0, 1, 0]
------------------------
clause index: 0
l: 1
operations count: 168
------------------------
clause index: 0
l: 2
operations count: 169
------------------------
clause index: 1
l: 3
operations count: 170
ls: 1.7894736842105265
------------------------
------------------------
clause index to compute satisfactory assignment: 0
satisfactory assignment: [1, 1, 0, 0, 1, 1]
------------------------
clause index: 1
l: 1
operations count: 171
ls: 1.7812500000000002
------------------------
------------------------
clause index to compute satisfactory assignment: 1
satisfactory assignment: [0, 0, 0, 0, 1, 0]
------------------------
clause index: 0
l: 1
operations count: 172
------------------------
clause index: 1
l: 

In [13]:
# randomly generated instance
KLI1 = KLI(instance1, n, eps, debug=False)

print('------------------------')
print('approximation:', KLI1)
print('eps:', eps)
print('ratio (real eps):', (KL1/sol1)-1)

------------------------
approximation: 1086508.0
eps: 0.1
ratio (real eps): -0.007348060607910156
